In [9]:
import data_analisy as da
import data_cleaner as dc
import matplotlib as mp
import pandas as pd

# Leitura e tratamento da base de dados

Antes de qualquer coisa, precisamos usar as funções do modulo **data_cleaner** para ler e tratar os dados:

In [10]:
dataframe_without_tratament = dc.csv_to_dataframe('censo_escolar_2021.csv')
dataframe_treated = dc.remove_colunas_irrelevantes(dataframe_without_tratament, 3600)

# Análise de dados 
Antes de qualquer coisa, é necessário realizar uma analise da na base de dados. Então, iniciaremos por ai.  

Para começar vamos analisar as medidas de resumo de algumas colunas.

### Análise da coluna "QT_EQUIP_TV"

Aqui, vamos começar analisando a coluna que descreve a **quantidade de equipamentos de TV**  existente na escola.

In [11]:
tv_measure_tendency = da.measure_tendency(dataframe_treated, 'QT_EQUIP_TV')
tv_measure_dipresion = da.measure_dispersion(dataframe_treated, 'QT_EQUIP_TV')

print(f'A média é: {tv_measure_tendency[0]}; \nA mediana é: {tv_measure_tendency[1]}; \nA moda é: {tv_measure_tendency[2]}')
print(f'O desvio padrão é: {tv_measure_dipresion[0]}; \nA variância é: {tv_measure_dipresion[1]}; \nO valor máximo é: {tv_measure_dipresion[2]}; \nO valor mínimo é: {tv_measure_dipresion[3]}; \nA amplitude é de: {tv_measure_dipresion[4]}')

A média é: 94.653; 
A mediana é: 0.0; 
A moda é: 0
O desvio padrão é: 2884.967; 
A variância é: 8323031.879; 
O valor máximo é: 88888; 
O valor mínimo é: 0; 
A amplitude é de: 88888


Já com essa análise, podemos observar um resultado estranho. Um valor máximo de **88888** televisores em uma escola é algo irreal. Então, devemos investigar melhor a ocorrência desse valor. Inicialmente, podemos supor algumas causas prováveis:  

* Erro de digitação;
* Um código criado para indicar algo;
* Valor coorespondente a outra coluna.
* Entre outras...

Nossa primeira abordagem para identificar o problema é descobrir quais são os valores que aparecem nessa base de dados, pois, além do 88888, podem aparecer outros valores extremos.

In [12]:
dataframe_treated['QT_EQUIP_TV'].unique()

array([    0,     1,    18,     2,     3,     9,     7,    10,     5,
          11,     4,     8,    15,     6, 88888,    21,    20,    17,
          13,    22,    14,    16,    12,    25,    19,    50,    23,
          32,    24,    45,    34,    26,    48,    64,    43,    30,
          35,    81,    42])

Esse é um resultado interessante. Muitos desses valores não condizem com a realidade da maior parte das escolas brasileiras, já que ter, por exemplo, uma escola 45 TVs,já não é algo tão alinhado com a realidade brasileira. Contudo, tudo dependa da frequencia do aparecimento desses valores, já que existir, em todo brasil, poucas escola com 40 televisões, não é algo impossivel. Junto a isso, podemos notar que por a mediana ser 0, mais da metade dos valores são 0, indicando que grande incidência de valores menores. Porém, por ter uma média em torno de 100, pode-se notar que esses outros valores, **principalmente o 88888** aparecem com uma frequência considerável.  
Então, vamos analisar a frequência desses valores:


In [13]:
print(dataframe_treated['QT_EQUIP_TV'].value_counts().to_dict())

{0: 41724, 1: 23242, 2: 8732, 3: 3114, 4: 1464, 5: 754, 6: 381, 7: 257, 8: 204, 10: 152, 9: 109, 88888: 85, 11: 79, 12: 75, 13: 41, 14: 33, 20: 31, 16: 25, 15: 19, 17: 15, 18: 13, 19: 11, 22: 8, 21: 7, 25: 5, 23: 4, 24: 3, 50: 2, 32: 2, 45: 2, 34: 2, 26: 2, 30: 2, 48: 1, 64: 1, 43: 1, 35: 1, 81: 1, 42: 1}


Assim, podemos notar que todos esses valores se tratam de **outliers**. Podemos verificar isso usando os quartis posteriormente. Por agora, outra coisa interessante de se analisar, é que há a ocorrência de 85 valores 88888, o que descarta a possibilidade de ser um erro de digitação. Então, a próxima análise apropriada, é verificar o que consta no dicionário de dados.  
Com o dicionário de dados, conseguimos entender de onde vem o esse valor. Segundo consta no dicionário:  

<center>'88888 - registro com marcação de valor extremo (valor superior ao limite máximo de 4 equipamentos para cada 3 salas existentes - foram marcados apenas valores>3)'</center>

Ou seja, esses valores são grandes a ponto de não serem retratados, mas nem perto de 88888. Com isso, podemos escolher duas abordagem para tratar esse dado.  

1. Podemos simplesmente excluir as linhas com esses dados, já que são apenas 85 ocorrências;
2. Podemos substituir pelo maior valor retratado, mantendo a informação que se trata de um outlier.

Dessas, a que ha menor perda de informação é a segunda opção. Contudo iremos executar as duas para ver a diferença entre as abordagens.

In [14]:
serie_with_max_value = dataframe_treated.replace(88888, 81)
serie_value_deleted = dataframe_treated.drop(dataframe_treated[dataframe_treated['QT_EQUIP_TV'] == 88888 ].index)

tv_measure_tendency = da.measure_tendency(serie_value_deleted, 'QT_EQUIP_TV')
tv_measure_dipresion = da.measure_dispersion(serie_value_deleted, 'QT_EQUIP_TV')

print('Retirando os valores 88888 ficamos com:')
print(f'A média é: {tv_measure_tendency[0]}; \nA mediana é: {tv_measure_tendency[1]}; \nA moda é: {tv_measure_tendency[2]}')
print(f'O desvio padrão é: {tv_measure_dipresion[0]}; \nA variância é: {tv_measure_dipresion[1]}; \nO valor máximo é: {tv_measure_dipresion[2]}; \nO valor mínimo é: {tv_measure_dipresion[3]}; \nA amplitude é de: {tv_measure_dipresion[4]} \n')

tv_measure_tendency = da.measure_tendency(serie_with_max_value, 'QT_EQUIP_TV')
tv_measure_dipresion = da.measure_dispersion(serie_with_max_value, 'QT_EQUIP_TV')

print('Alterando os valores 88888 para 81 ficamos com:')
print(f'A média é: {tv_measure_tendency[0]}; \nA mediana é: {tv_measure_tendency[1]}; \nA moda é: {tv_measure_tendency[2]}')
print(f'O desvio padrão é: {tv_measure_dipresion[0]}; \nA variância é: {tv_measure_dipresion[1]}; \nO valor máximo é: {tv_measure_dipresion[2]}; \nO valor mínimo é: {tv_measure_dipresion[3]}; \nA amplitude é de: {tv_measure_dipresion[4]} \n')



Retirando os valores 88888 ficamos com:
A média é: 0.919; 
A mediana é: 0.0; 
A moda é: 0
O desvio padrão é: 1.732; 
A variância é: 2.999; 
O valor máximo é: 81; 
O valor mínimo é: 0; 
A amplitude é de: 81 

Alterando os valores 88888 para 81 ficamos com:
A média é: 1.004; 
A mediana é: 0.0; 
A moda é: 0
O desvio padrão é: 3.123; 
A variância é: 9.751; 
O valor máximo é: 81; 
O valor mínimo é: 0; 
A amplitude é de: 81 



Podemos ver que os valores da média são semelhantes, contudo, no desvio padrão e a variancia há uma grande diferença. Portanto, ficaremos com a estrategia com menos perca de informação entre as duas, que é substituindo os valores pelço maior valor da coluna.

In [15]:
dataframe_treated = dc.replace_88888_values(dataframe_treated)
dataframe_treated


/workspaces/a1_lp/data_cleaner.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column].replace(88888, int(max), inplace = True)


,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
0,2021,Norte,1,Rond⌠nia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021,Norte,1,Rond⌠nia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,0.0,0.0,0.0,0.0,11.0,7.0,4.0,5.0,5.0,0.0
2,2021,Norte,1,Rond⌠nia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021,Norte,1,Rond⌠nia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0
4,2021,Norte,1,Rond⌠nia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,1102,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80600,2021,Nordeste,2,Alagoas,AL,27,Macei≤,2704302,Leste Alagoano,2703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
80601,2021,Nordeste,2,Alagoas,AL,27,Macei≤,2704302,Leste Alagoano,2703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
80602,2021,Nordeste,2,Alagoas,AL,27,Macei≤,2704302,Leste Alagoano,2703,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80603,2021,Nordeste,2,Alagoas,AL,27,Macei≤,2704302,Leste Alagoano,2703,...,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
